---
title: Reproducibility Experiment for Carl et al. (2020)
subtitle: "ADD SUBTITLE"
date: 2025-09-03
keywords: ["machine learning", "reproducibility", "animal species classification", "computer vision", "neural networks", "cnn", "resnet", "tensorflow", "wildlife monitoring"]
exports: 
  - format: pdf
    template: arxiv_nips
---

+++ {"part": "abstract"}
This experiment attempts to reproduce the results of the paper [Automated detection of European wild mammal species in camera trap images with an existing and pre-trained computer vision model](doi:10.1007/s10344-020-01404-y), 
which tests the pretrained Google Inception-ResNet-v2 model for predicting animal species.
We describe the required software, image loading processes, model outputs. Furthermore we calculate prediction global and per-class prediction accuracies and compare them to the metrics from the original paper.
+++

# Dependencies

As the exact dependencies are not provided, state-of-the-art packages are chosen and installed and imported. The exact versions are shown in {numref}`table-requirements`.

In [ ]:
!python3.12 -m pip install -r requirements.txt

In [7]:
# image & data processing
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import tabulate
# neural network
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import decode_predictions
# evaluation
from sklearn.metrics import accuracy_score, confusion_matrix
# plot
import matplotlib.pyplot as plt
import seaborn as sns

# Model

In [2]:
model = InceptionResNetV2(weights="imagenet")

2025-09-04 02:20:47.017300: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Data
For the experiment, 90 common animals are used. They are sourced from Google images and provided in a labeled format in @banerjee2024animal. The Kaggle dataset is rather large. To mimic the original experiment setup, only 10 samples are used for each species.

In [3]:
DATA_PATH = Path("data/kaggle-90-different-animals")
model_input_size = model.input_shape[1:3] # the required image dimensions (299, 299)

## Data Preprocessing
The images are loaded with three color channels (RGB), resized to 299 by 299 pixels and converted into an 1-dimensional vector. The color intensities are scaled to be floating point numbers from 0 to 1. This is the minimal preprocessing required to fit the required input size of the neural network.

```{image} data/kaggle-90-different-animals/wombat/2a6c3fd292.jpg
:align: center
:alt: Example input image
```

In [4]:
def load_normalized_image(path, target_size=model_input_size):
    image = Image.open(path).convert("RGB")
    image = image.resize(target_size)
    return np.array(image) / 255.0

In [6]:
wildlife_image_paths = []

animal_species = sorted([d.name for d in DATA_PATH.iterdir() if d.is_dir()])

for species_name in animal_species:
    animal_image_folder = DATA_PATH / species_name # every species has its image folder
    for image_path in animal_image_folder.glob("*.jpg"):
        image_array = load_normalized_image(image_path)
        wildlife_image_paths.append(image_path)

The testing data is constructed by stacking the vectors and using the folder names as the label.

In [9]:
animal_images = [load_normalized_image(p) for p in wildlife_image_paths]
animal_species = [p.parent.name for p in wildlife_image_paths]

X_test = np.stack(animal_images, axis=0)
y_true = animal_species

# Test
To relate the output from the neural network to the labels from the dataset, only the output from the top neuron of the final softmax layer is used for each prediction. 

In [11]:
y_pred = model.predict(X_test)
y_pred = [pred[0][1] for pred in decode_predictions(y_pred, top=1)]

2025-09-04 02:26:26.949382: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 965530800 exceeds 10% of free system memory.


29/29 ━━━━━━━━━━━━━━━━━━━━ 111s 4s/step


When comparing the true labels and the predictions, it becomes apparent, that the model actually yields usable results. Almost all inference outputs are animal species somehow related to the one present in the image. This shows that the InceptionResNetV2 is generalizable to some extent.

In [14]:
species_recognition_result = pd.DataFrame({
    "y_true": y_true,
    "y_pred": y_pred
})
species_recognition_result

,y_true,y_pred
0,antelope,gazelle
1,antelope,impala
2,antelope,impala
3,antelope,gazelle
4,antelope,gazelle
...,...,...
895,zebra,zebra
896,zebra,zebra
897,zebra,zebra
898,zebra,zebra


## Label Mapping
One big issue with this experiment is the set of classes known to the model which do not match the dataset used for testing. To calculate some sensible performance metrics, the animal species labels need to be mapped first. Note that this results in lost semantic information, because multiple species are often mapped to one single family (e.g., American Black Bear and Brown Bear are both mapped to simply bear). View {numref}`table-imagenet-label-mapping` for details.

In [15]:
imagenet_to_kaggle = {
    "gazelle": "antelope",
    "impala": "antelope",
    "American_black_bear": "bear",
    "brown_bear": "bear",
    "ground_beetle": "beetle",
    "leaf_beetle": "beetle",
    "rhinoceros_beetle": "beetle",
    "dung_beetle": "beetle",
    "wild_boar": "boar",
    "ringlet": "butterfly",
    "monarch": "butterfly",
    "sulphur_butterfly": "butterfly",
    "lycaenid": "butterfly",
    "Egyptian_cat": "cat",
    "tabby": "cat",
    "Siamese_cat": "cat",
    "Persian_cat": "cat",
    "lynx": "cat",
    "ox": "cow",
    "water_buffalo": "cow",
    "Dungeness_crab": "crab",
    "red_deer": "deer",
    "elk": "deer",
    "Labrador_retriever": "dog",
    "Border_collie": "dog",
    "Chihuahua": "dog",
    "Bouvier_des_Flandres": "dog",
    "Brittany_spaniel": "dog",
    "English_setter": "dog",
    "Greater_Swiss_Mountain_dog": "dog",
    "Ibizan_hound": "dog",
    "Mexican_hairless": "dog",
    "Pekinese": "dog",
    "Pomeranian": "dog", 
    "golden_retriever": "dog",
    "pug": "dog",
    "ass": "donkey",
    "mallard": "duck",
    "bald_eagle": "eagle",
    "golden_eagle": "eagle",
    "African_elephant": "elephant",
    "Indian_elephant": "elephant",
    "Arctic_fox": "fox",
    "red_fox": "fox",
    "ibex": "goat",
    "mountain_goat": "goat",
    "Arabian_horse": "horse",
    "Appaloosa": "horse",
    "wallaby": "kangaroo",
    "agama": "lizard",
    "alligator_lizard": "lizard",
    "Komodo_dragon": "lizard",
    "American_lobster": "lobster",
    "house_mouse": "mouse",
}

species_recognition_result["y_pred_mapped"] = species_recognition_result["y_pred"].map(
    lambda l: imagenet_to_kaggle.get(l, l) # safe map accessor (labels that are not present in the dict keys, remain unchanged)
)

# Evalution

In [16]:
accuracy_score(species_recognition_result["y_true"], species_recognition_result["y_pred_mapped"])

0.4622222222222222

In [17]:
species_recognition_result.assign(correct = species_recognition_result["y_true"] == species_recognition_result["y_pred_mapped"]).groupby("y_true")["correct"].mean().sort_values(ascending=False)

y_true
bison         1.0
bear          1.0
boar          1.0
eagle         1.0
cockroach     1.0
             ... 
turkey        0.0
whale         0.0
turtle        0.0
wolf          0.0
woodpecker    0.0
Name: correct, Length: 90, dtype: float64

In [11]:
species_recognition_result[species_recognition_result["y_pred_mapped"] != species_recognition_result["y_true"]]

,y_true,y_pred,y_pred_mapped
6,antelope,ibex,goat
14,badger,American_black_bear,bear
20,bat,hummingbird,hummingbird
21,bat,wood_rabbit,wood_rabbit
22,bat,hook,hook
...,...,...,...
885,woodpecker,hornbill,hornbill
886,woodpecker,bittern,bittern
887,woodpecker,bittern,bittern
888,woodpecker,hornbill,hornbill


In [12]:
species_recognition_result[species_recognition_result["y_pred_mapped"] != species_recognition_result["y_true"]].to_csv("species_recognition_result2.csv", index=False)

# Summary

# Future Work

# Appendix

## Dependency Table

In [16]:
package_versions = {}
with open("requirements.txt", "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "==" in line:
            pkg, ver = line.split("==")
            package_versions[pkg] = ver
        else:
            package_versions[line] = "unknown"
            
with open("figure/requirements.md", "w") as f:
    f.write(
        pd.DataFrame({
            "package": package_versions.keys(),
            "version": package_versions.values()
        }).to_markdown(index=False)
    )

```{table} 
:name: table-requirements
:align: center

Runtime dependencies
```{include} figure/requirements.md

## Imagenet Label Mapping Table

In [17]:
with open("figure/imagenet_label_mapping.md", "w") as f:
    f.write(
        pd.DataFrame({
            "imagenet label": imagenet_to_kaggle.keys(),
            "mapped label": imagenet_to_kaggle.values()
        }).groupby("mapped label")["imagenet label"].apply(
            lambda labels: ", ".join(labels)
        ).reset_index().to_markdown(index=False)
    )

NameError: name 'imagenet_to_kaggle' is not defined

```{table} 
:name: table-imagenet-label-mapping
:align: center

Imagenet label mapping
```{include} figure/imagenet_label_mapping.md

## Inference Output Table

In [29]:
github_animal_image_paths = [
    f"[{p.parent.name}](https://github.com/tobsel7/research-vetmedwien-animal-species-identification/raw/main/{p})"
    for p in wildlife_image_paths
]

with open("figure/species_recognition_result.md", "w") as f:
    f.write(
        pd.DataFrame({
            "truth": github_animal_image_paths,
            "mapped prediction": species_recognition_result["y_pred_mapped"],
            "model prediction": species_recognition_result["y_pred"]
        }).to_markdown(index=False)
    )

```{table} 
:name: species-recognition-result
:align: center

Inception-ResNet-v2 predictions
```{include} figure/species_recognition_result.md